# Using the geojson generator

## Import the packages

In [1]:
import pandas as pd

In [2]:
from convert2geojson import Convert2GeoJson
from SPARQLWrapper import SPARQLWrapper, JSON

In [3]:
def runQuery(query):
    sparql = SPARQLWrapper(
        "http://localhost:8080/blazegraph/namespace/kb/sparql" # This is the workshop endpoint
    )
    sparql.setReturnFormat(JSON)                     # and we want json as returns
    sparql.setQuery(query)
    try:
        ret = sparql.queryAndConvert()
        return ret["results"]["bindings"]
    except Exception as e:
        print(e)
        return 

## 1) Load annotations that have coordinate data

In [24]:
linkedObjects = runQuery("""
prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
prefix oa: <http://www.w3.org/ns/oa#>
prefix geojson: <https://purl.org/geojson/vocab#>

SELECT  ?name ?anno ?coord1 ?coord2
    FROM <http://example.org/ns/users/malte/genesis-example-geo2>
WHERE { 
      ?anno a oa:Annotation ;
        oa:hasTarget / oa:hasSelector / oa:exact ?name .
      ?anno a oa:Annotation ;
        oa:hasBody / geojson:geometry / geojson:coordinates ?coords .
      ?coords rdf:first ?coord1 ;
          rdf:rest / rdf:first ?coord2 .
  }
"""
)


In [34]:
linkedObjects[:2]

[{'coord1': {'datatype': 'http://www.w3.org/2001/XMLSchema#double',
   'type': 'literal',
   'value': '14.32854'},
  'coord2': {'datatype': 'http://www.w3.org/2001/XMLSchema#double',
   'type': 'literal',
   'value': '35.9064'},
  'anno': {'type': 'uri',
   'value': 'https://recogito.pelagios.org/annotation/a6f6c0b4-f0e9-4040-a55e-519f7e0ba913'},
  'name': {'datatype': 'http://www.w3.org/2001/XMLSchema#string',
   'type': 'literal',
   'value': 'Ir'}},
 {'coord1': {'datatype': 'http://www.w3.org/2001/XMLSchema#double',
   'type': 'literal',
   'value': '44.403333'},
  'coord2': {'datatype': 'http://www.w3.org/2001/XMLSchema#double',
   'type': 'literal',
   'value': '14.211389'},
  'anno': {'type': 'uri',
   'value': 'https://recogito.pelagios.org/annotation/ec0b357e-8dbd-4412-8384-af56ce8f24b3'},
  'name': {'datatype': 'http://www.w3.org/2001/XMLSchema#string',
   'type': 'literal',
   'value': 'Sephar'}}]

In [56]:
geoDataframe = pd.DataFrame([[y[x]['value'] for x in ['coord1', 'coord2', 'anno', 'name']] for y in linkedObjects],columns=['latitude', 'longitude', 'annotation', 'name'])

In [73]:
df = pd.DataFrame([(float(tup[0]), float(tup[1]), tup[2], '<br/>'.join(g0.annotation.values)) for tup, g0 in geoDataframe.groupby(['latitude', 'longitude', 'name'])], columns=['latitude', 'longitude', 'name',  'annotation'])

In [74]:
df

,latitude,longitude,name,annotation
0,-73.329280,42.027310,Canaan,https://recogito.pelagios.org/annotation/c8ad7...
1,121.597990,17.038090,Eden,https://recogito.pelagios.org/annotation/d02a0...
2,14.328540,35.906400,Ir,https://recogito.pelagios.org/annotation/a6f6c...
3,17.090000,-23.317000,Rehoboth,https://recogito.pelagios.org/annotation/3e443...
4,34.500000,41.500000,Assyria,https://recogito.pelagios.org/annotation/c6538...
5,34.606980,31.381760,Gerar,https://recogito.pelagios.org/annotation/bbf00...
6,35.375640,33.559930,Sidon,https://recogito.pelagios.org/annotation/c8ae7...
7,44.403333,14.211389,Sephar,https://recogito.pelagios.org/annotation/ec0b3...
8,44.500000,31.500000,Euphrates,https://recogito.pelagios.org/annotation/4b119...
9,67.500000,42.500000,Gaza,https://recogito.pelagios.org/annotation/ad0d4...


### Build GeoJSON

Note that not every dataframe row contains information on longitude or latitude. These rows are dropped from the dataframe. Therefore, the number of found features may vary.

Instantiate using the dataframe, a list of properties to be displayed in popups, and the column titles for the geodata. By adding `.convert()` the geojson is created directly after instantiation.

In [75]:
build = Convert2GeoJson(dataframe=df, properties=df.columns, lat='latitude',lon='longitude').convert()

You can provide additional information for the geoJSON by adding an attribution (`attribution='Generated by ...'`) and a description (`description='Shows the ...`).


The generated geoJSON in dictionary format can be accessed by `.geojson()`.

In [76]:
geojson = build.geojson()
geojson.keys()

dict_keys(['type', 'features', 'properties'])

To save the generated geoJSON as a file, chose

In [ ]:
#build.save('./data/testGEO.geojson')

You can also provide parameters for the filename (`name='data.geojson'`) or path (`path='.'`).

### Display result

The package uses ipyleaflet to generate the map for the geoJSON data. 

Markers on maps can be drawn fully (standard) or grouped with leaflets `MarkerCluster` (style='grouped'). The basemap for the background mapping is loaded from CartoDBs 'base-light-nolabels' source. Additonal parameters include customized basemaps or layouting for the map.

In [80]:
mapD = build.display()

In [81]:
mapD

Map(center=[34.0573203, 26.6307879], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title…

## 2)  Added value

Find rivers that are linked by `P53_has_location` to a geolocated postition. Use this information to create a dataframe and geojson, that plots the tagged rivers on a map. 

In [82]:
landObjects = runQuery(
"""
 PREFIX crm: <http://cidoc-crm.org/cidoc-crm/>

    SELECT ?sub ?obj
        FROM <http://example.org/ns/users/malte/genesis-example-geo2>
    WHERE {
        ?sub crm:P53_has_location ?obj .
    }
"""
)

In [83]:
landObjects

[{'sub': {'type': 'uri',
   'value': 'http://example.org/ns/users/malte/genesis-example-geo2/places/Havilah'},
  'obj': {'type': 'uri',
   'value': 'http://example.org/ns/users/malte/genesis-example-geo2/rivers/Pishon'}},
 {'sub': {'type': 'uri',
   'value': 'http://example.org/ns/users/malte/genesis-example-geo2/places/Cush'},
  'obj': {'type': 'uri',
   'value': 'http://example.org/ns/users/malte/genesis-example-geo2/rivers/Gihon'}},
 {'sub': {'type': 'uri',
   'value': 'http://example.org/ns/users/malte/genesis-example-geo2/places/Assyria'},
  'obj': {'type': 'uri',
   'value': 'http://example.org/ns/users/malte/genesis-example-geo2/rivers/Hiddekel'}}]